In [ ]:
# Comparison between MLP-ANN and Analytic Method on 6-DOF

import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Function for the traditional calculation of inverse kinematics
def traditional_inverse_kinematics(px_val, py_val, pz_val, roll_val, pitch_val, yaw_val):
    # Euler Rotation Matrix for RPY
    R_roll = Matrix([[1, 0, 0],
                     [0, cos(roll), -sin(roll)],
                     [0, sin(roll), cos(roll)]])
    R_pitch = Matrix([[cos(pitch), 0, sin(pitch)],
                      [0, 1, 0],
                      [-sin(pitch), 0, cos(pitch)]])
    R_yaw = Matrix([[cos(yaw), -sin(yaw), 0],
                    [sin(yaw), cos(yaw), 0],
                    [0, 0, 1]])
    R_rpy = R_yaw * R_pitch * R_roll

    # Solve for wrist position
    R0_6 = R_rpy.subs({roll: roll_val, pitch: pitch_val, yaw: yaw_val})
    wrist_x = px_val - d6 * R0_6[0, 2]
    wrist_y = py_val - d6 * R0_6[1, 2]
    wrist_z = pz_val - d6 * R0_6[2, 2]

    # Solve for theta1
    theta1_sol = atan2(wrist_y, wrist_x)

    # Solve for theta2 and theta3
    r = sqrt(wrist_x**2 + wrist_y**2) - a1
    s = wrist_z - d1
    D = (r**2 + s**2 - a2**2 - a3**2) / (2 * a2 * a3)
    theta3_sol = atan2(-sqrt(1 - D**2), D)
    theta2_sol = atan2(s, r) - atan2(a3 * sin(theta3_sol), a2 + a3 * cos(theta3_sol))

    R0_3 = Matrix([[cos(theta1_sol) * cos(theta2_sol + theta3_sol), -sin(theta1_sol), -cos(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta1_sol) * cos(theta2_sol + theta3_sol), cos(theta1_sol), -sin(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta2_sol + theta3_sol), 0, cos(theta2_sol + theta3_sol)]])
    R3_6 = R0_3.inv("LU") * R0_6

    # Solve for theta4, theta5, theta6
    theta4_sol = atan2(R3_6[2, 2], -R3_6[0, 2])
    theta5_sol = atan2(sqrt(R3_6[0, 2]**2 + R3_6[2, 2]**2), R3_6[1, 2])
    theta6_sol = atan2(-R3_6[1, 1], R3_6[1, 0])

    return [theta1_sol, theta2_sol, theta3_sol, theta4_sol, theta5_sol, theta6_sol]

# Loding Data
df = pd.read_csv('',  encoding='utf8')
df = df.drop(['Unnamed: 0'], axis=1)
x_scaler = MinMaxScaler(copy=True, feature_range=(-1, 1))
y_scaler = MinMaxScaler(copy=True, feature_range=(-1, 1))
X = df.iloc[:, :12]
y = df.iloc[:, 12:]
X_s = x_scaler.fit_transform(X)
y_s = y_scaler.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2)

# MLP Model
NN_model = Sequential()
NN_model.add(Dense(12, kernel_initializer='normal', input_dim=X_train.shape[1], activation='relu'))
NN_model.add(Dense(200, kernel_initializer='normal', activation='relu'))
NN_model.add(Dense(200, kernel_initializer='normal', activation='relu'))
NN_model.add(Dense(6, kernel_initializer='normal', activation='linear'))
NN_model.compile(loss='mean_absolute_error', optimizer=Adam(), metrics=['accuracy'])
NN_model.fit(X_train, y_train, epochs=70, validation_split=0.2)

# Measuring execution time for MLP
start_time = time.time()
y_pred_mlp = NN_model.predict(X_test)
end_time = time.time()
mlp_execution_time = end_time - start_time

# Preparing input data for traditional (hypothetical) inverse kinematics
px_test, py_test, pz_test, roll_test, pitch_test, yaw_test = [X_test[:, i] for i in range(6)]

# Measuring execution time for the traditional method
start_time = time.time()
for i in range(len(px_test)):
    traditional_inverse_kinematics(px_test[i], py_test[i], pz_test[i], roll_test[i], pitch_test[i], yaw_test[i])
end_time = time.time()
traditional_execution_time = end_time - start_time

# Comparing execution times
print("MLP Execution Time:", mlp_execution_time, "seconds")
print("Traditional Method Execution Time:", traditional_execution_time, "seconds")


In [ ]:
# Comparison between LSTM-ANN and Analytic Method on 6-DOF

import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Function for the traditional calculation of inverse kinematics
def traditional_inverse_kinematics(px_val, py_val, pz_val, roll_val, pitch_val, yaw_val):
    # Euler Rotation Matrix for RPY
    R_roll = Matrix([[1, 0, 0],
                     [0, cos(roll), -sin(roll)],
                     [0, sin(roll), cos(roll)]])
    R_pitch = Matrix([[cos(pitch), 0, sin(pitch)],
                      [0, 1, 0],
                      [-sin(pitch), 0, cos(pitch)]])
    R_yaw = Matrix([[cos(yaw), -sin(yaw), 0],
                    [sin(yaw), cos(yaw), 0],
                    [0, 0, 1]])
    R_rpy = R_yaw * R_pitch * R_roll

    # Solve for wrist position
    R0_6 = R_rpy.subs({roll: roll_val, pitch: pitch_val, yaw: yaw_val})
    wrist_x = px_val - d6 * R0_6[0, 2]
    wrist_y = py_val - d6 * R0_6[1, 2]
    wrist_z = pz_val - d6 * R0_6[2, 2]

    # Solve for theta1
    theta1_sol = atan2(wrist_y, wrist_x)

    # Solve for theta2 and theta3
    r = sqrt(wrist_x**2 + wrist_y**2) - a1
    s = wrist_z - d1
    D = (r**2 + s**2 - a2**2 - a3**2) / (2 * a2 * a3)
    theta3_sol = atan2(-sqrt(1 - D**2), D)
    theta2_sol = atan2(s, r) - atan2(a3 * sin(theta3_sol), a2 + a3 * cos(theta3_sol))


    R0_3 = Matrix([[cos(theta1_sol) * cos(theta2_sol + theta3_sol), -sin(theta1_sol), -cos(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta1_sol) * cos(theta2_sol + theta3_sol), cos(theta1_sol), -sin(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta2_sol + theta3_sol), 0, cos(theta2_sol + theta3_sol)]])
    R3_6 = R0_3.inv("LU") * R0_6

    # Solve for theta4, theta5, theta6
    theta4_sol = atan2(R3_6[2, 2], -R3_6[0, 2])
    theta5_sol = atan2(sqrt(R3_6[0, 2]**2 + R3_6[2, 2]**2), R3_6[1, 2])
    theta6_sol = atan2(-R3_6[1, 1], R3_6[1, 0])

    return [theta1_sol, theta2_sol, theta3_sol, theta4_sol, theta5_sol, theta6_sol]

# Loding Data
df = pd.read_csv('',  encoding='utf8')

df = df.drop(['Unnamed: 0'], axis=1)
scaler = MinMaxScaler(copy=True, feature_range=(-1, 1))
X = df.iloc[:, :12]
y = df.iloc[:, 12:]
X_scaler = scaler.fit(X)
X_s = scaler.transform(X)
y_scaler = scaler.fit(y)
y_s = scaler.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2)

# Formatting for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# LSTM Model
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(LSTM(256, activation='relu', return_sequences=True))
model.add(LSTM(256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='linear'))
model.compile(loss='mae', optimizer=Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=70, validation_split=0.2)

# Measuring execution time for LSTM
start_time = time.time()
y_pred_lstm = model.predict(X_test)
end_time = time.time()
lstm_execution_time = end_time - start_time

# Preparing input data for traditional (hypothetical) inverse kinematics
px_test, py_test, pz_test, roll_test, pitch_test, yaw_test = [X_test[:, 0, i] for i in range(6)]

# Measuring execution time for the traditional method
start_time = time.time()
for i in range(len(px_test)):
    traditional_inverse_kinematics(px_test[i], py_test[i], pz_test[i], roll_test[i], pitch_test[i], yaw_test[i])
end_time = time.time()
traditional_execution_time = end_time - start_time

# Comparing execution times
print("LSTM Execution Time:", lstm_execution_time, "seconds")
print("Traditional Method Execution Time:", traditional_execution_time, "seconds")


In [ ]:
# Comparison between BiLSTM-ANN and Analytic Method on 6-DOF

import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM
import matplotlib.pyplot as plt

# Function for the traditional calculation of inverse kinematics
def traditional_inverse_kinematics(px_val, py_val, pz_val, roll_val, pitch_val, yaw_val):
    # Euler Rotation Matrix for RPY
    R_roll = Matrix([[1, 0, 0],
                     [0, cos(roll), -sin(roll)],
                     [0, sin(roll), cos(roll)]])
    R_pitch = Matrix([[cos(pitch), 0, sin(pitch)],
                      [0, 1, 0],
                      [-sin(pitch), 0, cos(pitch)]])
    R_yaw = Matrix([[cos(yaw), -sin(yaw), 0],
                    [sin(yaw), cos(yaw), 0],
                    [0, 0, 1]])
    R_rpy = R_yaw * R_pitch * R_roll

    # Solve for wrist position
    R0_6 = R_rpy.subs({roll: roll_val, pitch: pitch_val, yaw: yaw_val})
    wrist_x = px_val - d6 * R0_6[0, 2]
    wrist_y = py_val - d6 * R0_6[1, 2]
    wrist_z = pz_val - d6 * R0_6[2, 2]

    # Solve for theta1
    theta1_sol = atan2(wrist_y, wrist_x)

    # Solve for theta2 and theta3
    r = sqrt(wrist_x**2 + wrist_y**2) - a1
    s = wrist_z - d1
    D = (r**2 + s**2 - a2**2 - a3**2) / (2 * a2 * a3)
    theta3_sol = atan2(-sqrt(1 - D**2), D)
    theta2_sol = atan2(s, r) - atan2(a3 * sin(theta3_sol), a2 + a3 * cos(theta3_sol))


    R0_3 = Matrix([[cos(theta1_sol) * cos(theta2_sol + theta3_sol), -sin(theta1_sol), -cos(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta1_sol) * cos(theta2_sol + theta3_sol), cos(theta1_sol), -sin(theta1_sol) * sin(theta2_sol + theta3_sol)],
                   [sin(theta2_sol + theta3_sol), 0, cos(theta2_sol + theta3_sol)]])
    R3_6 = R0_3.inv("LU") * R0_6

    # Resolver para theta4, theta5, theta6
    theta4_sol = atan2(R3_6[2, 2], -R3_6[0, 2])
    theta5_sol = atan2(sqrt(R3_6[0, 2]**2 + R3_6[2, 2]**2), R3_6[1, 2])
    theta6_sol = atan2(-R3_6[1, 1], R3_6[1, 0])

    return [theta1_sol, theta2_sol, theta3_sol, theta4_sol, theta5_sol, theta6_sol]


# Loding Data
df = pd.read_csv('',  encoding='utf8')
df = df.drop(['Unnamed: 0'], axis=1)
scaler = MinMaxScaler(copy=True, feature_range=(-1, 1))
X = df.iloc[:, :12]
y = df.iloc[:, 12:]
X_scaler = scaler.fit(X)
X_s = scaler.transform(X)
y_scaler = scaler.fit(y)
y_s = scaler.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2)

# Formatting para Bi-LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Bi-LSTM Model
model = Sequential()
model.add(Bidirectional(LSTM(32, activation='relu', return_sequences=True), input_shape=(X_train.shape[1:])))
model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(256, activation='relu')))
model.add(Dense(6, activation='linear'))
model.compile(loss='mae', optimizer='Adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=70, validation_split=0.2)

# Measuring execution time for BiLSTM
start_time = time.time()
y_pred_bilstm = model.predict(X_test)
end_time = time.time()
bilstm_execution_time = end_time - start_time

# Preparing input data for traditional (hypothetical) inverse kinematics
px_test, py_test, pz_test, roll_test, pitch_test, yaw_test = [X_test[:, 0, i] for i in range(6)]

# Measuring execution time for the traditional method
start_time = time.time()
for i in range(len(px_test)):
    traditional_inverse_kinematics(px_test[i], py_test[i], pz_test[i], roll_test[i], pitch_test[i], yaw_test[i])
end_time = time.time()
traditional_execution_time = end_time - start_time

# Comparing execution times
print("BiLSTM Execution Time:", bilstm_execution_time, "seconds")
print("Traditional Method Execution Time:", traditional_execution_time, "seconds")
